In [1]:
from lm_polygraph.utils import UEManager

/home/nkotelevskii/github/multidimensional_uncertainty/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
man_cal = UEManager.load('../resources/llm_resources/llama8b_xsum.man') # просто чтобы напечатать, какие меры неопределённости есть
for el in man_cal.estimations.keys():
    print(el)

/home/nkotelevskii/github/multidimensional_uncertainty/.venv/lib/python3.12/site-packages/lm_polygraph/utils/manager.py:705: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  re

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Stat calculators: [<lm_polygraph.stat_calculators.greedy_probs.GreedyProbsCalculator object at 0x7fdbde91a390>]
('sequence', 'GreedySemanticEnrichedPPLAveDissimilarity')
('sequence', 'GreedySemanticEnrichedMaxprobAveDissimilarity')
('sequence', 'GreedySemanticEnrichedMTEAveDissimilarity')
('sequence', 'BestSemanticEnrichedPPLAveDissimilarity')
('sequence', 'BestSemanticEnrichedMaxprobAveDissimilarity')
('sequence', 'BestSemanticEnrichedMTEAveDissimilarity')
('sequence', 'MbrSemanticEnrichedPPLAveDissimilarity')
('sequence', 'MbrSemanticEnrichedMaxprobAveDissimilarity')
('sequence', 'MbrSemanticEnrichedMTEAveDissimilarity')
('sequence', 'SemanticEntropy')
('sequence', 'SAR_t0.001')
('sequence', 'MaximumSequenceProbability')
('sequence', 'Perplexity')
('sequence', 'MeanTokenEntropy')
('sequence', 'BestSampledMaximumSequenceProbability')
('sequence', 'BestSampledPerplexity')
('sequence', 'BestSampledMeanTokenEntropy')
('sequence', 'MbrSampledMaximumSequenceProbability')
('sequence', 'MbrS

In [ ]:
from mdu.unc.entropic_ot import EntropicOTOrdering
from mdu.unc.constants import OTTarget, SamplingMethod, ScalingType
import numpy as np

In [ ]:
# это можно не менять
hyperparams_list = [
    {
        'target': OTTarget.EXP,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.FEATURE_WISE,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.EXP,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.GLOBAL,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.BETA,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.FEATURE_WISE,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
    {
        'target': OTTarget.BETA,
        'sampling_method': SamplingMethod.GRID,
        'scaling_type': ScalingType.GLOBAL,
        'grid_size': 5,
        'n_targets_multiplier': 1,
        'eps': 0.5,
        'max_iters': 1000,
        'tol': 1e-6,
    },
]

In [ ]:
calib_ratio = 0.25 # это доля объектов, которая будет использована для калибровки (обучения ОТ). Её можно менять
import pickle

for model_name in ["llama8b", "falcon7b", "mistral7b"]:

    file_names = [
        f'{model_name}_xsum.man',
        f'{model_name}_wmt14_fren.man',
        f'{model_name}_wmt19_deen.man',
        f'{model_name}_trivia.man',
        f'{model_name}_mmlu.man',
        f'{model_name}_gsm8k_cot.man',
        f'{model_name}_coqa_no_context.man',
    ]

    for file_name in file_names:
        man_current = UEManager.load(file_name)

        measures_dict = {k: v for k, v in man_current.estimations.items()}

        # Это я вручную создаю CoCoA меры (я проглядел, что они уже были в исходном менеджере, но под другими именами)
        measures_dict[('sequence', 'CoCoA_greedy_MSP')] = man_current.estimations[('sequence', 'GreedyAveDissimilarity')] * man_current.estimations[('sequence', 'MaximumSequenceProbability')]
        measures_dict[('sequence', 'CoCoA_greedy_PPL')] = man_current.estimations[('sequence', 'GreedyAveDissimilarity')] * man_current.estimations[('sequence', 'Perplexity')]
        measures_dict[('sequence', 'CoCoA_greedy_MTE')] = man_current.estimations[('sequence', 'GreedyAveDissimilarity')] * man_current.estimations[('sequence', 'MeanTokenEntropy')]

        measures_dict[('sequence', 'CoCoA_best_MSP')] = man_current.estimations[('sequence', 'BestAveDissimilarity')] * man_current.estimations[('sequence', 'BestSampledMaximumSequenceProbability')]
        measures_dict[('sequence', 'CoCoA_best_PPL')] = man_current.estimations[('sequence', 'BestAveDissimilarity')] * man_current.estimations[('sequence', 'BestSampledPerplexity')]
        measures_dict[('sequence', 'CoCoA_best_MTE')] = man_current.estimations[('sequence', 'BestAveDissimilarity')] * man_current.estimations[('sequence', 'BestSampledMeanTokenEntropy')]

        n_samples = len(man_current.estimations[('sequence', 'Perplexity')])
        np.random.seed(42)
        # Это я фиксирую калибровочную маску
        calib_mask = np.random.choice(n_samples, size=int(n_samples * calib_ratio), replace=False)

        final_measures = {}

        final_measures['calibration_mask'] = calib_mask

        for hyperparams in hyperparams_list:
            target = hyperparams['target']
            sampling_method = hyperparams['sampling_method']
            scaling_type = hyperparams['scaling_type']
            grid_size = hyperparams['grid_size']
            n_targets_multiplier = hyperparams['n_targets_multiplier']
            eps = hyperparams['eps']
            max_iters = hyperparams['max_iters']
            tol = hyperparams['tol']


            model = EntropicOTOrdering(
                target=target,
                sampling_method=sampling_method,
                scaling_type=scaling_type,
                grid_size=grid_size,
                target_params={},
                eps=eps,
                n_targets_multiplier=n_targets_multiplier,
                max_iters=max_iters,
                random_state=42,
                tol=tol,
            )

            combinations = [
                (
                    f"CoCoA_like_1_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'GreedyAveDissimilarity'),
                        ('sequence', 'BestAveDissimilarity'),
                        ('sequence', 'BestSampledMaximumSequenceProbability'),
                        ('sequence', 'BestSampledPerplexity'),
                        ('sequence', 'BestSampledMeanTokenEntropy'),
                        ('sequence', 'MaximumSequenceProbability'),
                        ('sequence', 'Perplexity'),
                        ('sequence', 'MeanTokenEntropy'),
                    )
                ),
                (
                    f"CoCoA_like_2_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'BestAveDissimilarity'),
                        ('sequence', 'BestSampledMaximumSequenceProbability'),
                        ('sequence', 'BestSampledPerplexity'),
                        ('sequence', 'BestSampledMeanTokenEntropy'),
                    )
                ),
                (
                    f"CoCoA_like_3_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'GreedyAveDissimilarity'),
                        ('sequence', 'MaximumSequenceProbability'),
                        ('sequence', 'Perplexity'),
                        ('sequence', 'MeanTokenEntropy'),
                    )
                ),
                (
                    f"Master_1_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                    )
                ),
                (
                    f"Master_1_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                    )
                ),
                (
                    f"Master_2_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_best_MSP'),
                        ('sequence', 'CoCoA_best_PPL'),
                        ('sequence', 'CoCoA_best_MTE'),
                    )
                ),
                (
                    f"Master_3_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_best_MSP'),
                        ('sequence', 'CoCoA_best_PPL'),
                        ('sequence', 'CoCoA_best_MTE'),
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                    )
                ),
                (
                    f"IMBA_1_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_best_MSP'),
                        ('sequence', 'CoCoA_best_PPL'),
                        ('sequence', 'DegMat_NLI_score_entail'),
                    )
                ),
                (
                    f"IMBA_2_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                        ('sequence', 'DegMat_NLI_score_entail'),  # это имя, под которым будет сохраняться композитная мера
                    )
                ),
                (
                    f"IMBA_3_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_best_MSP'),
                        ('sequence', 'CoCoA_best_PPL'),
                        ('sequence', 'CoCoA_best_MTE'),
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                        ('sequence', 'DegMat_NLI_score_entail'),
                        ('sequence', 'EigValLaplacian_NLI_score_entail'),
                    )
                ),
                (
                    f"IMBA_4_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                        ('sequence', 'DegMat_NLI_score_entail'),
                        ('sequence', 'MaximumSequenceProbability'),
                    )
                ),
                (
                    f"IMBA_5_{target.value}_{scaling_type.value}",  # это имя, под которым будет сохраняться композитная мера 
                    (# а дальше в этом кортеже просто напихиваешь те unc меры, которые хочешь объединить
                        ('sequence', 'CoCoA_greedy_MSP'),
                        ('sequence', 'CoCoA_greedy_PPL'),
                        ('sequence', 'CoCoA_greedy_MTE'),
                        ('sequence', 'MaximumSequenceProbability'),
                    )
                ),
            ]

            for comb_list in combinations:
                name, comb = comb_list[0], comb_list[1]
                components = [measures_dict[el] for el in comb]
                
                calib_components = [np.array(comp)[calib_mask] for comp in components]

                stacked_calib_components = np.column_stack(calib_components)
                stacked_test_components = np.column_stack(components)

                model.fit(stacked_calib_components)
                scores = model.predict(stacked_test_components)

                final_measures[name] = scores

        with open(f'../resources/llm_resources/final_measures_{file_name.split(".")[0]}.pkl', 'wb') as f:
            pickle.dump(final_measures, f)

In [1]:
import pickle
from lm_polygraph.utils import UEManager

models = ['llama8b', 'falcon7b','mistral7b']
datasets = ['trivia', 'mmlu','wmt14_fren','wmt19_deen','xsum','gsm8k_cot','coqa_no_context']
results = []
quality_metrics={
    'trivia':'AlignScoreOutputTarget',
    'mmlu':'Accuracy',
    'wmt14_fren':'Comet',
    'wmt19_deen':'Comet',
    'xsum':'AlignScoreInputOutput',
    'gsm8k_cot':'Accuracy',
    'coqa_no_context':'AlignScoreOutputTarget'
}
for model in models: 
    for dataset in datasets:
        man = UEManager.load(f'../resources/llm_resources/{model}_{dataset}.man')

        with open(f'../resources/llm_resources/final_measures_{model}_{dataset}.pkl', 'rb') as f:
            final_measures_1 = pickle.load(f)
        for method in list(final_measures_1.keys()):
            if method!='calibration_mask':
                man.estimations[('sequence', method)] = final_measures_1[method]
        
        calibration_mask = final_measures_1['calibration_mask']
        for key, values in man.estimations.items():
            man.estimations[key] = [v for i, v in enumerate(values) if i not in calibration_mask]
        
        for key, values in man.gen_metrics.items():
            man.gen_metrics[key] = [v for i, v in enumerate(values) if i not in calibration_mask]

        from lm_polygraph.ue_metrics import PredictionRejectionArea

        ue_metrics =[PredictionRejectionArea(max_rejection=0.5)]


        man.ue_metrics=ue_metrics

        man.eval_ue()
        quality_metric =quality_metrics[dataset]
        methods = [x[1] for x in list(man.estimations.keys())]

        methods = [
            method
            for method in methods
            if "mbr" not in method.lower() and "sample" not in method.lower() and "best" not in method.lower()
        ]

        
        for method in methods:
            prr_score = man.metrics[('sequence', method, quality_metric, 'prr_0.5_normalized')]
            results.append({ 'model': model, 'Method':method, 'prr': prr_score, 'dataset':dataset})

/home/nkotelevskii/github/multidimensional_uncertainty/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import pandas as pd

# assume df has columns: ["Method", "dataset", "prr"]
df = pd.DataFrame(results)


tables = {}
for model, df_model in df.groupby("model"):
    df_wide = df_model.pivot_table(
        index="Method", columns="dataset", values="prr", aggfunc="mean"
    )
    df_wide["mean"] = df_wide.mean(axis=1)
    df_wide = df_wide.sort_values("mean", ascending=False).reset_index()
    tables[model] = df_wide
    filename = f"../resources/llm_resources/{model}_results.csv"
    df_wide.to_csv(filename, index=False)

# Example: show the table for one model
tables["llama8b"]

dataset,Method,coqa_no_context,gsm8k_cot,mmlu,trivia,wmt14_fren,wmt19_deen,xsum,mean
0,IMBA_5_Exp_FeatureWise,0.353907,0.419594,0.484351,0.597100,0.475661,0.612408,0.405422,0.478349
1,IMBA_5_Beta_FeatureWise,0.355931,0.409402,0.481497,0.599280,0.475688,0.613109,0.409612,0.477789
2,Master_1_Exp_FeatureWise,0.356833,0.438419,0.461269,0.605990,0.470549,0.565129,0.408392,0.472369
3,Master_1_Beta_FeatureWise,0.357576,0.430522,0.460514,0.606566,0.470590,0.564865,0.410731,0.471623
4,Master_1_Exp_Global,0.368312,0.366505,0.463711,0.608272,0.459690,0.595384,0.400275,0.466021
5,Master_1_Beta_Global,0.368916,0.365808,0.463258,0.608094,0.458775,0.595235,0.400645,0.465819
6,GreedySemanticEnrichedMaxprobAveDissimilarity,0.366889,0.362325,0.491508,0.602602,0.442589,0.584028,0.394897,0.463548
7,IMBA_2_Beta_Global,0.376108,0.368005,0.461837,0.614290,0.440997,0.577845,0.400559,0.462806
8,IMBA_2_Exp_Global,0.377028,0.371017,0.461118,0.616403,0.438409,0.573779,0.401458,0.462745
9,IMBA_4_Exp_FeatureWise,0.379458,0.430302,0.478185,0.619169,0.344328,0.563956,0.410475,0.460839


In [4]:
import pandas as pd

# assume df has columns: ["model", "Method", "dataset", "prr"]
df = pd.DataFrame(results)

tables = {}
for model, df_model in df.groupby("model"):
    # pivot to wide form
    df_wide = df_model.pivot_table(
        index="Method", columns="dataset", values="prr", aggfunc="mean"
    )

    # add mean across datasets
    df_wide["mean"] = df_wide.mean(axis=1)

    # compute ranking for each dataset (higher prr = better rank)
    for col in df_wide.columns:
        if col != "mean":
            df_wide[f"{col}_rank"] = df_wide[col].rank(
                method="min", ascending=False
            ).astype(int)

    # also rank by mean
    df_wide["mean_rank"] = df_wide["mean"].rank(
        method="min", ascending=False
    ).astype(int)

    # sort by mean rank
    df_wide = df_wide.sort_values("mean", ascending=False).reset_index()

    tables[model] = df_wide
    filename = f"../resources/llm_resources/{model}_results.csv"
    df_wide.to_csv(filename, index=False)

# Example: show the table for one model
tables["llama8b"][:15]

dataset,Method,coqa_no_context,gsm8k_cot,mmlu,trivia,wmt14_fren,wmt19_deen,xsum,mean,coqa_no_context_rank,gsm8k_cot_rank,mmlu_rank,trivia_rank,wmt14_fren_rank,wmt19_deen_rank,xsum_rank,mean_rank
0,IMBA_5_Exp_FeatureWise,0.353907,0.419594,0.484351,0.597100,0.475661,0.612408,0.405422,0.478349,28,10,9,35,2,2,8,1
1,IMBA_5_Beta_FeatureWise,0.355931,0.409402,0.481497,0.599280,0.475688,0.613109,0.409612,0.477789,27,15,11,30,1,1,5,2
2,Master_1_Exp_FeatureWise,0.356833,0.438419,0.461269,0.605990,0.470549,0.565129,0.408392,0.472369,23,1,28,19,4,19,6,3
3,Master_1_Beta_FeatureWise,0.357576,0.430522,0.460514,0.606566,0.470590,0.564865,0.410731,0.471623,22,4,30,18,3,20,2,4
4,Master_1_Exp_Global,0.368312,0.366505,0.463711,0.608272,0.459690,0.595384,0.400275,0.466021,18,25,24,15,5,5,13,5
5,Master_1_Beta_Global,0.368916,0.365808,0.463258,0.608094,0.458775,0.595235,0.400645,0.465819,16,26,25,16,6,6,11,6
6,GreedySemanticEnrichedMaxprobAveDissimilarity,0.366889,0.362325,0.491508,0.602602,0.442589,0.584028,0.394897,0.463548,19,28,3,25,9,9,14,7
7,IMBA_2_Beta_Global,0.376108,0.368005,0.461837,0.614290,0.440997,0.577845,0.400559,0.462806,8,23,27,14,11,12,12,8
8,IMBA_2_Exp_Global,0.377028,0.371017,0.461118,0.616403,0.438409,0.573779,0.401458,0.462745,7,22,29,10,15,15,9,9
9,IMBA_4_Exp_FeatureWise,0.379458,0.430302,0.478185,0.619169,0.344328,0.563956,0.410475,0.460839,5,5,14,8,39,21,3,10


In [5]:
import pandas as pd

# assume df has columns: ["model", "Method", "dataset", "prr"]
df = pd.DataFrame(results)

tables = {}
for model, df_model in df.groupby("model"):
    # pivot to wide form
    df_wide = df_model.pivot_table(
        index="Method", columns="dataset", values="prr", aggfunc="mean"
    )

    # add mean across datasets
    df_wide["mean"] = df_wide.mean(axis=1)

    # build a new DataFrame with MultiIndex columns: (dataset, "score") and (dataset, "rank")
    df_nice = {}
    for col in df_wide.columns:
        if col != "mean":
            df_nice[(col, "score")] = df_wide[col]
            df_nice[(col, "rank")] = df_wide[col].rank(method="min", ascending=False).astype(int)

    # add mean score + rank
    df_nice[("mean", "score")] = df_wide["mean"]
    df_nice[("mean", "rank")] = df_wide["mean"].rank(method="min", ascending=False).astype(int)

    # convert dict to DataFrame
    df_pretty = pd.DataFrame(df_nice, index=df_wide.index)

    # sort by mean rank
    df_pretty = df_pretty.sort_values(("mean", "rank")).reset_index()

    tables[model] = df_pretty
    filename = f"../resources/llm_resources/{model}_results.csv"
    df_pretty.to_csv(filename)

# Example: show table for one model
tables["llama8b"]

Method coqa_no_context       \
                                                            score rank   
0                          IMBA_5_Exp_FeatureWise        0.353907   28   
1                         IMBA_5_Beta_FeatureWise        0.355931   27   
2                        Master_1_Exp_FeatureWise        0.356833   23   
3                       Master_1_Beta_FeatureWise        0.357576   22   
4                             Master_1_Exp_Global        0.368312   18   
5                            Master_1_Beta_Global        0.368916   16   
6   GreedySemanticEnrichedMaxprobAveDissimilarity        0.366889   19   
7                              IMBA_2_Beta_Global        0.376108    8   
8                               IMBA_2_Exp_Global        0.377028    7   
9                          IMBA_4_Exp_FeatureWise        0.379458    5   
10                  CoCoA_like_3_Beta_FeatureWise        0.336145   43   
11                        IMBA_4_Beta_FeatureWise        0.379941    4   
12                   CoCoA_like_3_Exp_FeatureWise        0.339954   42   
13      GreedySemanticEnrichedPPLAveDissimilarity        0.351629   31   
14                         IMBA_2_Exp_FeatureWise        0.382852    2   
15                        IMBA_2_Beta_FeatureWise        0.380607    3   
16      GreedySemanticEnrichedMTEAveDissimilarity        0.350511   33   
17                      Master_3_Beta_FeatureWise        0.356386   25   
18                       Master_3_Exp_FeatureWise        0.356415   24   
19                  CoCoA_like_1_Beta_FeatureWise        0.340199   40   
20                   CoCoA_like_1_Exp_FeatureWise        0.340182   41   
21                              IMBA_4_Exp_Global        0.350788   32   
22                            Master_3_Exp_Global        0.362935   21   
23                             IMBA_4_Beta_Global        0.348072   34   
24                           Master_3_Beta_Global        0.364434   20   
25                              IMBA_5_Exp_Global        0.343138   38   
26                             IMBA_5_Beta_Global        0.341805   39   
27                        IMBA_3_Beta_FeatureWise        0.376074    9   
28                         IMBA_3_Exp_FeatureWise        0.377913    6   
29                             IMBA_3_Beta_Global        0.375223   11   
30                              IMBA_3_Exp_Global        0.373297   13   
31                        CoCoA_like_3_Exp_Global        0.314715   50   
32                       CoCoA_like_3_Beta_Global        0.315034   49   
33                        CoCoA_like_1_Exp_Global        0.321302   47   
34                     MaximumSequenceProbability        0.290338   53   
35                       CoCoA_like_1_Beta_Global        0.317172   48   
36                      Master_2_Beta_FeatureWise        0.346188   37   
37                       Master_2_Exp_FeatureWise        0.346868   35   
38                              IMBA_1_Exp_Global        0.370339   14   
39                             IMBA_1_Beta_Global        0.368736   17   
40                                     Perplexity        0.260974   55   
41                            Master_2_Exp_Global        0.353573   29   
42                           Master_2_Beta_Global        0.353239   30   
43                  CoCoA_like_2_Beta_FeatureWise        0.330824   45   
44                   CoCoA_like_2_Exp_FeatureWise        0.333256   44   
45                                     SAR_t0.001        0.323909   46   
46                         GreedyAveDissimilarity        0.391002    1   
47                        IMBA_1_Beta_FeatureWise        0.374696   12   
48                         IMBA_1_Exp_FeatureWise        0.376030   10   
49                               MeanTokenEntropy        0.247590   57   
50                                       CEDegMat        0.356052   26   
51                        CoCoA_like_2_Exp_Global        0.302724   52   
52                       CoCoA_like_2_Beta_Glob

In [6]:
import pandas as pd

# assume df has columns: ["model", "Method", "dataset", "prr"]
df = pd.DataFrame(results)

TOP_K = 5

# 1) Aggregate to one score per (model, dataset, method)
agg = (
    df.groupby(["model", "dataset", "Method"], as_index=False)["prr"]
      .mean()
)

# 2) Rank methods within each (model, dataset), highest prr = rank 1
agg["rank"] = (
    agg.groupby(["model", "dataset"])["prr"]
       .rank(method="min", ascending=False)
       .astype(int)
)

# 3) Keep only top K
topk = agg[agg["rank"] <= TOP_K].copy()

# 4) Build a nested dict: { model: { dataset: DataFrame(...) } }
topk_tables = {}
for model, df_m in topk.groupby("model"):
    model_dict = {}
    for ds, df_ds in df_m.groupby("dataset"):
        model_dict[ds] = (
            df_ds[["Method", "prr", "rank"]]
            .sort_values(["rank", "prr"], ascending=[True, False])
            .reset_index(drop=True)
        )
    topk_tables[model] = model_dict

# 5) Example: print top-5 for one model across all datasets
model = "llama8b"
for ds in sorted(topk_tables[model].keys()):
    print(f"\n=== {model} :: {ds} — Top {TOP_K} ===")
    print(topk_tables[model][ds].to_string(index=False))

# 6) (Optional) Save each dataset’s top-5 per model to CSVs
for model, ds_dict in topk_tables.items():
    for ds, df_ds in ds_dict.items():
        out = f"../resources/llm_resources/{model}_{ds}_top{TOP_K}.csv"
        df_ds.to_csv(out, index=False)


=== llama8b :: coqa_no_context — Top 5 ===
                 Method      prr  rank
 GreedyAveDissimilarity 0.391002     1
 IMBA_2_Exp_FeatureWise 0.382852     2
IMBA_2_Beta_FeatureWise 0.380607     3
IMBA_4_Beta_FeatureWise 0.379941     4
 IMBA_4_Exp_FeatureWise 0.379458     5

=== llama8b :: gsm8k_cot — Top 5 ===
                                   Method      prr  rank
                 Master_1_Exp_FeatureWise 0.438419     1
                   IMBA_2_Exp_FeatureWise 0.435330     2
GreedySemanticEnrichedMTEAveDissimilarity 0.433381     3
                Master_1_Beta_FeatureWise 0.430522     4
                   IMBA_4_Exp_FeatureWise 0.430302     5

=== llama8b :: mmlu — Top 5 ===
                                       Method      prr  rank
                   MaximumSequenceProbability 0.516374     1
                            IMBA_5_Exp_Global 0.492882     2
GreedySemanticEnrichedMaxprobAveDissimilarity 0.491508     3
                           IMBA_5_Beta_Global 0.491450     4
    